In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.decomposition import NMF
#from movies_titles import ratings



1. nmf_recommender

In [2]:
df = pd.read_csv('/Users/Varvara/spiced_working_files/cleaned_df.csv', index_col=0)

#create R matrix
ratings =df.pivot_table(index='userId', columns='title', values='rating')
ratings.fillna(ratings.mean().mean(),inplace=True)
 
#create dictionaries
unique_movies = df['title'].unique()
movie_list = df['title'].tolist()
movies_dict = {unique_movies[i] : i for i in range(len(unique_movies))}

movie_title_dict={key:value for (value,key) in enumerate(movies_dict)}
movie_title_dict

{'Toy Story (1995)': 0,
 'Grumpier Old Men (1995)': 1,
 'Heat (1995)': 2,
 'Seven (a.k.a. Se7en) (1995)': 3,
 'Usual Suspects, The (1995)': 4,
 'From Dusk Till Dawn (1996)': 5,
 'Bottle Rocket (1996)': 6,
 'Braveheart (1995)': 7,
 'Rob Roy (1995)': 8,
 'Canadian Bacon (1995)': 9,
 'Desperado (1995)': 10,
 'Billy Madison (1995)': 11,
 'Clerks (1994)': 12,
 'Dumb & Dumber (Dumb and Dumber) (1994)': 13,
 'Ed Wood (1994)': 14,
 'Star Wars: Episode IV - A New Hope (1977)': 15,
 'Pulp Fiction (1994)': 16,
 'Stargate (1994)': 17,
 'Tommy Boy (1995)': 18,
 'Clear and Present Danger (1994)': 19,
 'Forrest Gump (1994)': 20,
 'Jungle Book, The (1994)': 21,
 'Mask, The (1994)': 22,
 'Blown Away (1994)': 23,
 'Dazed and Confused (1993)': 24,
 'Fugitive, The (1993)': 25,
 'Jurassic Park (1993)': 26,
 'Mrs. Doubtfire (1993)': 27,
 "Schindler's List (1993)": 28,
 'So I Married an Axe Murderer (1993)': 29,
 'Three Musketeers, The (1993)': 30,
 'Tombstone (1993)': 31,
 'Dances with Wolves (1990)': 32,
 

In [3]:
query = {'Silence of the Lambs, The (1991)': 5, 'Fight Club (1999)': 5,
         'The Butterfly Effect (2004)': 4,
         '(500) Days of Summer (2009)': 5,
         "Schindler's List (1993)": 5,}

In [4]:
count = 0
new_query = {}
    # construct a user vector
for movie in ratings.columns:
    if movie not in query:
        new_query[movie] = np.nan
    else:
        new_query[movie] = query[movie]

new_query = pd.DataFrame(new_query, index=[0])
new_query = np.array(new_query).reshape(1, -1)

#print(new_query)

user_dataframe = pd.DataFrame(new_query, index=[
                                  'Recommendation'], columns=ratings.columns).fillna(0)
print(user_dataframe)


title           '71 (2014)  'Hellboy': The Seeds of Creation (2004)  \
Recommendation         0.0                                      0.0   

title           'Round Midnight (1986)  'Salem's Lot (2004)  \
Recommendation                     0.0                  0.0   

title           'Til There Was You (1997)  'Tis the Season for Love (2015)  \
Recommendation                        0.0                              0.0   

title           'burbs, The (1989)  'night Mother (1986)  \
Recommendation                 0.0                   0.0   

title           (500) Days of Summer (2009)  *batteries not included (1987)  \
Recommendation                          5.0                             0.0   

title           ...  Zulu (2013)  [REC] (2007)  [REC]² (2009)  \
Recommendation  ...          0.0           0.0            0.0   

title           [REC]³ 3 Génesis (2012)  \
Recommendation                      0.0   

title           anohana: The Flower We Saw That Day - The Movie (2013)  \
R

In [5]:
m = NMF(20)
m = m.fit(ratings)
Q = m.components_  
P = m.transform(ratings) 

/Users/Varvara/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1692: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [6]:
pickle.dump(m,open("nmf_model1.sav", "wb"))

In [7]:
model = pickle.load(open("nmf_model1.sav", "rb"))
model.components_

array([[ 2.77967679,  0.44846492,  0.48760734, ...,  0.17320855,
         2.50312146,  0.48769405],
       [19.61543974, 50.26075398, 58.29988159, ..., 48.9395024 ,
        96.58254268, 31.88967576],
       [25.35409837, 42.98329341, 24.00025509, ..., 70.63653224,
        36.8067817 , 24.62666902],
       ...,
       [ 2.02011762,  2.15378533,  2.38844194, ...,  1.87569136,
         1.83347171,  2.38442346],
       [ 2.21561505,  1.58982712,  1.58490773, ...,  1.5801697 ,
         2.09558202,  1.58708984],
       [ 2.5879948 ,  3.17394958,  3.17595606, ...,  3.16935865,
         2.56955089,  3.18743819]])

In [8]:
# calculate the score with the NMF model
user_P = model.transform(user_dataframe)
Q = model.components_
user_R = np.dot(user_P, Q)
user_dataframe = pd.DataFrame(
        user_R, index=['Recommendation'], columns=ratings.columns)

In [9]:
user_P

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.68212524e-04,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 3.29199471e-05, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [10]:
user_R

array([[0.0102511 , 0.00696115, 0.00594001, ..., 0.00338517, 0.01238052,
        0.00738162]])

In [11]:
user_dataframe

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
Recommendation,0.010251,0.006961,0.00594,0.004305,0.004527,0.006829,0.020911,0.006701,0.029979,0.008231,...,0.010725,0.003274,0.012202,0.006361,0.006549,0.023902,0.010228,0.003385,0.012381,0.007382


In [12]:
unrated_boolean = np.isnan(new_query)[0]
unrated_df = user_dataframe.iloc[:, unrated_boolean]
sorted_new_user_df = unrated_df.T.sort_values(
        by='Recommendation', ascending=False).head(10)

recommendations10= sorted_new_user_df.head(10)
print((list(recommendations10.index)))

['Pulp Fiction (1994)', 'Matrix, The (1999)', 'Godfather, The (1972)', 'Shawshank Redemption, The (1994)', 'Usual Suspects, The (1995)', 'Reservoir Dogs (1992)', 'Seven (a.k.a. Se7en) (1995)', 'Saving Private Ryan (1998)', 'American Beauty (1999)', 'American History X (1998)']


In [13]:
def recommend_nmf(query, model='nmf_model.sav', k=10):
    """Filters and recommends the top k movies 
    for any given input query based 
    on a trained NMF model.

    Parameters
    ----------
    query : dict
        A dictionary of movies already seen. 
        Takes the form {"movie_A": 3, "movie_B": 3} etc
    nmf_model : pickle
        pickle nmf_model read from disk
    k : int, optional
        no. of top movies to recommend, by default 10
    """
    # 1. candiate generation
    count = 0
    new_query = {}
    # construct a user vector
    for movie in ratings.columns:

        if movie not in query:

            new_query[movie] = np.nan
            # new_query.append(fuzzy[0])
        else:
            new_query[movie] = query[movie]

    new_query = pd.DataFrame(new_query, index=[0])
    new_query = np.array(new_query).reshape(1, -1)

    #print(new_query)

    user_dataframe = pd.DataFrame(new_query, index=[
                                  'Recommendation'], columns=ratings.columns).fillna(0)
    #print(user_dataframe)

    # 2. scoring

    # calculate the score with the NMF nmf_model
    model = pickle.load(open("nmf_model.sav", "rb"))
    # calculate the score with the NMF model
    user_P = model.transform(user_dataframe)
    Q = model.components_
    user_R = np.dot(user_P, Q)

    user_dataframe = pd.DataFrame(
        user_R, index=['Recommendation'], columns=ratings.columns)

    # 3. ranking

    # set zero score to movies already seen by the user
    unrated_boolean = np.isnan(new_query)[0]
    unrated_df = user_dataframe.iloc[:, unrated_boolean]
    sorted_new_user_df = unrated_df.T.sort_values(
        by='Recommendation', ascending=False).head(k)

    recommendations10= sorted_new_user_df.head(k)
    return list(recommendations10.index)

2. cf_recommender

In [14]:
from scipy.spatial import distance

In [15]:
ratings=ratings.T

In [16]:
cf_array = distance.squareform(distance.pdist(ratings, metric='cosine'))
cf_array

array([[0.00000000e+00, 8.37359284e-05, 5.05526781e-05, ...,
        9.30532309e-04, 1.78944327e-03, 4.35565729e-04],
       [8.37359284e-05, 0.00000000e+00, 2.35729937e-05, ...,
        7.29883806e-04, 1.78944327e-03, 4.35565729e-04],
       [5.05526781e-05, 2.35729937e-05, 0.00000000e+00, ...,
        6.97189743e-04, 1.75658528e-03, 4.02592862e-04],
       ...,
       [9.30532309e-04, 7.29883806e-04, 6.97189743e-04, ...,
        0.00000000e+00, 2.44172695e-03, 1.08683970e-03],
       [1.78944327e-03, 1.78944327e-03, 1.75658528e-03, ...,
        2.44172695e-03, 0.00000000e+00, 2.14374776e-03],
       [4.35565729e-04, 4.35565729e-04, 4.02592862e-04, ...,
        1.08683970e-03, 2.14374776e-03, 0.00000000e+00]])

In [17]:
cf_matrix = pd.DataFrame(cf_array) 
cf_matrix

,0,1,2,3,4,5,6,7,8,9,...,9709,9710,9711,9712,9713,9714,9715,9716,9717,9718
0,0.000000,0.000084,0.000051,0.000274,0.000279,0.000281,0.001933,0.000047,0.003949,0.000152,...,0.000281,0.000707,0.000142,0.000082,0.000047,0.001656,0.002286,0.000931,0.001789,0.000436
1,0.000084,0.000000,0.000024,0.000274,0.000279,0.000281,0.001933,0.000047,0.003976,0.000152,...,0.000281,0.000790,0.000169,0.000052,0.000047,0.001656,0.002142,0.000730,0.001789,0.000436
2,0.000051,0.000024,0.000000,0.000241,0.000246,0.000248,0.001891,0.000014,0.003942,0.000119,...,0.000248,0.000756,0.000136,0.000019,0.000014,0.001622,0.002110,0.000697,0.001757,0.000403
3,0.000274,0.000274,0.000241,0.000000,0.000005,0.000470,0.002122,0.000237,0.004169,0.000342,...,0.000470,0.000981,0.000359,0.000242,0.000237,0.001849,0.002328,0.000918,0.001978,0.000625
4,0.000279,0.000279,0.000246,0.000005,0.000000,0.000476,0.002128,0.000242,0.004173,0.000347,...,0.000476,0.000986,0.000365,0.000247,0.000242,0.001853,0.002335,0.000924,0.001984,0.000630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9714,0.001656,0.001656,0.001622,0.001849,0.001853,0.001845,0.003567,0.001646,0.005953,0.001665,...,0.001845,0.002379,0.001743,0.001620,0.001616,0.000000,0.003580,0.002279,0.003643,0.001998
9715,0.002286,0.002142,0.002110,0.002328,0.002335,0.002346,0.003973,0.002108,0.005657,0.001945,...,0.001729,0.003572,0.002261,0.002053,0.002108,0.003580,0.000000,0.001803,0.003784,0.002502
9716,0.000931,0.000730,0.000697,0.000918,0.000924,0.000931,0.002582,0.000695,0.004857,0.000799,...,0.000931,0.002143,0.000879,0.000629,0.000695,0.002279,0.001803,0.000000,0.002442,0.001087
9717,0.001789,0.001789,0.001757,0.001978,0.001984,0.001989,0.003491,0.001743,0.005433,0.001739,...,0.001917,0.002489,0.001874,0.001759,0.001754,0.003643,0.003784,0.002442,0.000000,0.002144


In [24]:
query

{'Silence of the Lambs, The (1991)': 5,
 'Fight Club (1999)': 5,
 'The Butterfly Effect (2004)': 4,
 '(500) Days of Summer (2009)': 5,
 "Schindler's List (1993)": 5}

In [29]:
from fuzzywuzzy import process

/Users/Varvara/opt/anaconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [30]:
for m in query:
        fuzzy = process.extractOne(m, movies_dict.keys())              
        loc_sim = movie_title_dict[fuzzy[0]]  

In [41]:
loc_sim

28

In [42]:
neighbour_movies = cf_matrix.loc[loc_sim].sort_values(ascending=False)  
neighbour_movies

6865    0.023502
5512    0.023155
7421    0.020839
3158    0.019997
7593    0.019770
          ...   
4860    0.000122
8047    0.000122
32      0.000108
33      0.000019
28      0.000000
Name: 28, Length: 9719, dtype: float64

In [40]:
top3 = neighbour_movies.sort_values(ascending=False)[:3]
top3 = top3.to_frame()
top3

,28
6865,0.023502
5512,0.023155
7421,0.020839


In [56]:
top3=pd.DataFrame(top3)

In [61]:
type(top3)

pandas.core.frame.DataFrame

In [62]:
top3.index

Int64Index([6865, 5512, 7421], dtype='int64')

In [68]:
int(top3.index[1])

5512

In [69]:
recommendations = []
for m in range(0, 3):
    movie_title = movie_title_dict[int(top3.index[m])]
    recommendations.append(movie_title)

    print(recommendations)

KeyError: 6865